In [0]:
%%sh
aws s3 ls  s3://b2c-prod-dca-store-estimates/iap_revenue/date=2021-12-12/report_type=FIANL/

In [0]:

iap_path = "s3://b2c-prod-dca-store-estimates/iap_revenue/version=3.0.0/"
# spark.read.parquet(iap_path).createOrReplaceTempView("test")

# spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/iap_revenue/").parquet("s3://b2c-prod-dca-store-estimates/iap_revenue/date=*/report_type=*").filter("date between '2021-11-20' and '2021-12-04' and report_type='PREVIEW' ").where("isSubscription ='true'").show(100, False)
spark.read.option("basePath", "s3://b2c-prod-dca-store-estimates/iap_revenue/version=3.0.0/").parquet("s3://b2c-prod-dca-store-estimates/iap_revenue/version=3.0.0/date=*/report_type=*").filter("date = '2022-02-20' and report_type='FINAL' ").createOrReplaceTempView("test")
# spark.sql("select count(1) from test where app_id=284882215 and country_code='US' limit 10").show(10, False)
# spark.sql("select country_code,sku,app_id,count(distinct(isSubscription)) as co from test group by country_code,sku,app_id  order by  co desc limit 10").show(100, False)
spark.sql("select * from test where sku='com.facebook.Facebook.fan_funding.10155900505621075' limit 100").show(100, False)
# spark.sql("select sum(est_iap_revenue), sum(est_iap_vol) from test where app_id =1455487903  and country_code = 'US'  limit 100").show(100, False)



In [0]:

iap_path = "s3://b2c-prod-dca-store-estimates/iap_revenue/date=2021-11-15/report_type=FINAL/"
# spark.read.parquet(iap_path).createOrReplaceTempView("test")
# spark.sql("select app_id, sum(est_iap_revenue) as iap_revenue, sum(est_iap_vol) as iap_vol from test where  app_id = 284862083 and country_code = 'GB' group by app_id limit 100").show(100,  False)
# spark.sql("select * from test limit 1").show(10, False)
# spark.sql("select count(1) from test where app_id=284862083 and country_code='GB'  limit 100").show(100, False)
# spark.sql("select count(1) from test limit 10").show(10)




import datetime
import pandas as pd

def get_date_list(begin_date, end_date, freq):
    date_list = [x.strftime('%Y-%m-%d') for x in list(pd.date_range(start=begin_date, end=end_date, freq=freq))]
    return date_list
    


def get_date_granularity_mapping_list(begin_date, end_date):
    # date_granularity_mapping_list = {
    #     "weekly": get_date_list(begin_date, end_date, "W-SAT")
    # }
    date_granularity_mapping_list = {
        "daily": get_date_list(begin_date, end_date, "D")
    }
    return date_granularity_mapping_list

begin_date = datetime.datetime.strptime("2022-02-23", '%Y-%m-%d')
# end_date = datetime.datetime.strptime("2021-11-15", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2022-02-26", '%Y-%m-%d')


DATE_DICT = get_date_granularity_mapping_list(begin_date, end_date)
# range_type_mapping = {"daily":"DAY","monthly":"MONTH","weekly":"WEEK"}



for granularity in DATE_DICT:
    for date in DATE_DICT[granularity]:        
        path =  "s3://b2c-prod-dca-store-estimates/iap_revenue/version=3.0.0/date={date}/report_type=PREVIEW/".format(date=date)
        spark.read.parquet(path).createOrReplaceTempView("test")
        result = spark.sql("SELECT count(1) FROM test ").collect()
        # result = spark.sql("SELECT sum(est_iap_revenue), sum(est_iap_vol)  FROM test ").collect()
        # result = spark.sql("SELECT date, count(distinct unified_product_id,country_code,device_code,market_code,granularity,date)  FROM test group by date order by date desc ").collect()
        print(date, result[0][0])
        

In [0]:
 

from pyspark.sql.functions import *
from data_lib.api import spark_loader
from data_lib.api.data_op import compare_columns_and_get_result_df
from data_lib.api import Conv

## Load wide df & Then convert to Apache arrow df and do some comparison

loader = spark_loader(spark)
p_path = "s3://b2c-prod-data-pipeline-qa/fredric/retention_data_2021_10_31_wide/"

## this one is raw data in S3
wide_df = loader.load_parquet_as_table(p_path, "retention_wide")


def get_store_iap_est(date):
    path =  "s3://b2c-prod-dca-store-estimates/iap_revenue/version=3.0.0/date={date}/report_type=FINAL/".format(date=date)
    spark.read.parquet(path).createOrReplaceTempView("test")
    result = spark.sql("SELECT count(1) FROM test ").collect()
    return date, result[0][0]
    
def get_snowflake_data(date, table_name):
    sf_df = loader.load_sf_as_table("select count(1) from {table_name} where date={date} ".format(table_name=table_name, date=date), "snowflake_table")
    return sf_df.count()

## Snowfake data here
sf_df = loader.load_sf_as_table("select count(1) from AA_INTELLIGENCE_PRODUCTION.ADL_STORE_PAID.FACT_STORE_IAP_EST_RESTATE_V1_CLUSTER_BY_PRODUCT_KEY where date='2021-11-15' ", "snowflake_table")
print(sf_df.count())